# K-mer comparisons!

This is a Jupyter Notebook using Python 3. You can use Shift-ENTER to run cells, and double click on code cells to edit them.

Contact: C. Titus Brown, ctbrown@ucdavis.edu

## Calculating Jaccard similarity and containment

Given any two collections of k-mers, we can calculate similarity and containment using the union and intersection functionality in Python.

In [ ]:
def jaccard_similarity(a, b):
    a = set(a)
    b = set(b)
    
    intersection = len(a.intersection(b))
    union = len(a.union(b))
    
    return intersection / union

In [ ]:
def jaccard_containment(a, b):
    a = set(a)
    b = set(b)
    
    intersection = len(a.intersection(b))
    
    return intersection / len(a)

## Let's try these functions out on some simple examples!

In [ ]:
a = ['ATGG', 'AACC']
b = ['ATGG', 'CACA']
c = ['ATGC', 'CACA']

In [ ]:
jaccard_similarity(a, a)

In [ ]:
jaccard_containment(a, a)

In [ ]:
jaccard_similarity(b, a)

In [ ]:
jaccard_similarity(a, c)

In [ ]:
jaccard_containment(b, a)

In [ ]:
%matplotlib inline
from matplotlib_venn import venn2, venn3

venn2([set(a), set(b)])

In [ ]:
venn3([set(a), set(b), set(c)])

## Calculating k-mers from DNA sequences

To extract k-mers from DNA sequences, we walk over the sequence with a sliding window:

In [ ]:
def build_kmers(sequence, ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1
    
    for i in range(n_kmers):
        kmer = sequence[i:i + ksize]
        kmers.append(kmer)
        
    return kmers

In [ ]:
build_kmers('ATGGACCAGATATAGGGAGAGCCAGGTAGGACA', 21)

In the k-mers that are output, you can see how the sequence shifts to the right - look at the pattern in the middle.

So, now, you can compare two sequences!

In [ ]:
seq1 = 'ATGGACCAGATATAGGGAGAGCCAGGTAGGACA'
seq2 = 'ATGGACCAGATATTGGGAGAGCCGGGTAGGACA'
# differences:       ^         ^

In [ ]:
K = 10
kmers1 = build_kmers(seq1, K)
kmers2 = build_kmers(seq2, K)

print(K, jaccard_similarity(kmers1, kmers2))

## Exercise: what happens to Jaccard similarity as you increase or decrease K?

In [ ]:
K = 10
kmers1 = build_kmers(seq1, K)
kmers2 = build_kmers(seq2, K)

print(K, jaccard_similarity(kmers1, kmers2))

## Reading k-mers in from a file

In practice, we often need to work with 100s of thousands of k-mers, and this means loading them in from sequences in files.

There are three cut-down genome files in the `genomes/` directory that we will use below:

```
akkermansia.fa
shew_os185.fa
shew_os223.fa
```
The latter two are two strains of *Shewanella baltica*, and the first one is an unrelated genome *Akkermansia muciniphila*.

In [ ]:
import screed # a library for reading in FASTA/FASTQ

def read_kmers_from_file(filename, ksize):
    all_kmers = []
    for record in screed.open(filename):
        sequence = record.sequence
        
        kmers = build_kmers(sequence, ksize)
        all_kmers += kmers

    return all_kmers

In [ ]:
akker_kmers = read_kmers_from_file('genomes/akkermansia.fa', 31)

In [ ]:
akker_kmers[:5]

In [ ]:
print(len(akker_kmers))

In [ ]:
shew1_kmers = read_kmers_from_file('genomes/shew_os185.fa', 31)
shew2_kmers = read_kmers_from_file('genomes/shew_os223.fa', 31)

We can see the relationship between these three like so:

In [ ]:
print('akker vs shew1', jaccard_similarity(akker_kmers, shew1_kmers))
print('akker vs shew2', jaccard_similarity(akker_kmers, shew2_kmers))
print('shew1 vs shew2', jaccard_similarity(shew1_kmers, shew2_kmers))

In [ ]:
print('akker vs shew1', jaccard_containment(akker_kmers, shew1_kmers))
print('akker vs shew2', jaccard_containment(akker_kmers, shew2_kmers))
print('shew1 vs shew2', jaccard_containment(shew1_kmers, shew2_kmers))

In [ ]:
venn3([set(akker_kmers), set(shew1_kmers), set(shew2_kmers)])

## Exercise: how do the above numbers change when you change ksize?

In [ ]:
K = 31
shew1_test = read_kmers_from_file('genomes/shew_os185.fa', K)
shew2_test = read_kmers_from_file('genomes/shew_os223.fa', K)

print('similarity, shew1 vs shew2', jaccard_similarity(shew1_test, shew2_test))
print('containment, shew1 vs shew2', jaccard_containment(shew1_test, shew2_test))

# Let's hash!

## Choose a hash function!

We need to pick a hash function that takes DNA k-mers and converts them into numbers.

Both the [mash](https://mash.readthedocs.io/en/latest/) software for MinHash, and the [sourmash](https://sourmash.readthedocs.io) software for modulo and MinHash, use MurmurHash:

https://en.wikipedia.org/wiki/MurmurHash

this is implemented in the 'mmh3' library in Python.

The other thing we need to do here is take into account the fact that DNA is double stranded, and so

```
hash_kmer('ATGG')
```
should be equivalent to
```
hash_kmer('CCAT')
```
Following mash's lead, for every input k-mer we will choose a *canonical* k-mer that is the lesser of the k-mer and its reverse complement.

In [ ]:
import mmh3

def hash_kmer(kmer):
    # calculate the reverse complement
    rc_kmer = screed.rc(kmer)
    
    # determine whether original k-mer or reverse complement is lesser
    if kmer < rc_kmer:
        canonical_kmer = kmer
    else:
        canonical_kmer = rc_kmer
        
    # calculate murmurhash using a hash seed of 42
    hash = mmh3.hash64(canonical_kmer, 42)[0]
    if hash < 0: hash += 2**64
        
    # done
    return hash

This is now a function that we can use to turn any DNA "word" into a number:

In [ ]:
hash_kmer('ATGGC')

The same input word always returns the same number:

In [ ]:
hash_kmer('ATGGC')

as does its reverse complement:

In [ ]:
hash_kmer('GCCAT')

and nearby words return very different numbers:

In [ ]:
hash_kmer('GCCAA')

## Note that hashing collections of k-mers doesn't change Jaccard calculations:

In [ ]:
def hash_kmers(kmers):
    hashes = []
    for kmer in kmers:
        hashes.append(hash_kmer(kmer))
    return hashes

In [ ]:
shew1_hashes = hash_kmers(shew1_kmers)
shew2_hashes = hash_kmers(shew2_kmers)

In [ ]:
print(jaccard_similarity(shew1_kmers, shew2_kmers))

In [ ]:
print(jaccard_similarity(shew1_hashes, shew2_hashes))

(ok, it changes it a little, because of the canonical k-mer calculation!)

## Implementing subsampling with modulo hashing

We are now ready to implement k-mer subsampling with modulo hash.

We need to pick a sampling rate, and know the maximum possible hash value.

For a sampling rate, let's start with 1000.

The MurmurHash function turns k-mers into numbers between 0 and `2**64 - 1` (the maximum 64-bit number).

Let's define these as variables:

In [ ]:
scaled = 1000
MAX_HASH = 2**64

Now, choose the range of hash values that we'll keep.

In [ ]:
keep_below = MAX_HASH / scaled
print(keep_below)

and write a filter function:

In [ ]:
def subsample_modulo(kmers):
    keep = []
    for kmer in kmers:
        if hash_kmer(kmer) < keep_below:
            keep.append(kmer)
        # otherwise, discard
        
    return keep

## Now let's apply this to our big collections of k-mers!

In [ ]:
akker_sub = subsample_modulo(akker_kmers)
shew1_sub = subsample_modulo(shew1_kmers)
shew2_sub = subsample_modulo(shew2_kmers)

In [ ]:
print(len(akker_kmers), len(akker_sub))
print(len(shew1_kmers), len(shew1_sub))
print(len(shew2_kmers), len(shew2_sub))

So we go from ~500,000 k-mers to ~500 hashes! Do the Jaccard calculations change??

In [ ]:
print('akker vs akker, total', jaccard_similarity(akker_kmers, akker_kmers))
print('akker vs akker, sub', jaccard_similarity(akker_sub, akker_sub))

In [ ]:
print('akker vs shew1, total', jaccard_similarity(akker_kmers, shew1_kmers))
print('akker vs shew1, sub', jaccard_similarity(akker_sub, shew1_sub))

In [ ]:
print('shew1 vs shew2, total', jaccard_similarity(shew1_kmers, shew2_kmers))
print('shew1 vs shew2, sub', jaccard_similarity(shew1_sub, shew2_sub))

And you can see that the numbers are different, but not very much - the Jaccard similarity is being *estimated*, so it is not exact but it is close.

## Let's visualize --

In [ ]:
venn3([set(akker_kmers), set(shew1_kmers), set(shew2_kmers)])

In [ ]:
venn3([set(akker_sub), set(shew1_sub), set(shew2_sub)])

## Exercise: does all this work for Jaccard containment, too?